In [17]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
LLM_MODEL_NAME="llama3:8b"
llm_model = ChatOllama(model=LLM_MODEL_NAME)
chain = llm_model | StrOutputParser()
chain.invoke("Hello")

e:\agenticai\venv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
messages = [
                ("system","You are a helpful AI assistant"),
                ("user","My name is {name}"),
                ("user","Whats my name?"),
            ]
prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template|llm_model | StrOutputParser()
chain.invoke({"name":"yash"})


'Nice to meet you, Yash! Your name is indeed Yash. How can I assist you today, friend?'

In [21]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm_model.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [22]:
config = {"configurable": {"thread_id": "abc123"}}
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Nice to meet you, Bob! How's it going?


In [23]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!
